In [1]:
import geopandas as gpd
import laspy
import numpy as np
import copy
import open3d as o3d
import pickle
from scipy.spatial import cKDTree
from sklearn.cluster import DBSCAN
import re
from interruptingcow import timeout
import os

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


Note that this notebook serves as both step 2 and step 4 of the framework. This notebook is applied after running notebook 1 and another time after running notebook 3 but with different input polygons. The required input is desribed at the end of the notebook overview. Do not forget to set the origin of the polygons in the settings cell that prompts you to do so.

# Notebook overview
This notebook completes three different step. In order to not have to save the really large point cloud files and carry them over to another notebook, the steps are combined into one notebook.

### Step 1
Firstly, we determine which point cloud files are required. The point clouds are cut into 50 by 50 meter squares and saved based on their coordinates. By looking at the coordinates of each crosswalk polygon in our data, we can determine in which point cloud file(s) they fall. As we sometimes need to grow elements of the crosswalk polygon to capture the entire crosswalk, we add bordering point cloud files for crosswalks that are close to the border of a point cloud file. We filter for the files that we need in order to not have to process all point cloud data, which would take a lot of time and computer power.

### Step 2
Once we know which point cloud files we will need to process the polygons, we continue by cutting and dowsnampling them. We cut the point clouds to only contain points on the ground as these are the only relevant ones for the task at hand. Furthermore, we downsample the point clouds in order to save computational resources and time, as the point cloud files are rather large.

### Step 3
When the point clouds are ready, we can match the crosswalk polygons to the point cloud data. We do this by creating a dictionary and adding an item for each crosswalk. This dictionary will contain all the point cloud points that fall within the polygon. 

### Step 4
Finally, we will process the point cloud data within each polygon. We first filter out all points that have an intensity value below a certain threshold as these are likely not part of road markings which have a large intensity value. The remaining points within a polygon will be clustered. This is done in order to isolate individual road markings (such as a signle crosswalk stripe). Each resulting cluster will be processed in order to capture a full feature, as sometimes Tile2Net only captures part of a crosswalk stripe. This is done by iteratively analyzing all bordering points of a cluster and adding the ones that exceed the intensity threshold, as these points are likely part of the same feature. This iterative process is repeated until no bordering points exceed the intensity threshold. This step results in a list of lists where each sub-list contains dictionaries of processed clusters that stem from the same original polygon.

### If coming from notebook 1
**Input**: 
- Shapely file with crosswalk polygons in area of interest.
- Location of folder that contains point cloud files. 

**Output**: 
- List of lists in which each sub-list contains dictionaries of processed clusters that stem from the same original polygon.

**Previous notebook**: 1. T2N processing.

**Next notebook**: 3. Polygon analysis + growing.

### If coming from notebook 3

**Input**: 
- Shapely file with extensions of processed crosswalk polygons.
- Location of folder that contains point cloud files. 

**Output**: 
- List of lists in which each sub-list contains dictionaries of processed clusters that stem from the same extension polygon as created in notebook 3.

**Previous notebook**: 3. Polygon analysis + growing.

**Next notebook**: 4. Extended polygon filtering.



## Settings

In [2]:
# Set CRS
CRS = 'epsg:28992'

In [3]:
# Determine whether you are running the code for the Tile2Net polygons (created in notebook 1) or the extension polygons (created in notebook 3)
# Choose "T2N" if coming from notebook 1 and "extension" if coming from notebook 3
polygon_origin = "T2N"

# Loading data

In [4]:
# Load crosswalk polygons
if polygon_origin == "T2N":
    CW_polygons = gpd.read_file("../data/output/crosswalk polygons Tile2Net.shp")
if polygon_origin == "extension":
    CW_polygons = gpd.read_file("../data/output/extended polygons.shp")

In [5]:
# Function to get the PC file names
def get_PC_files(folder):
    
    # Initiate list to save file coordinates
    file_list = []
    file_names = []

    # Get file names
    files = os.listdir(folder)

    # Pattern to filter out integers
    pattern = r'\d+'

    for file_name in files:

        # Search for .laz files
        match = re.search("\.laz$", file_name)

        if match:
            integers = re.findall(pattern, file_name)
            file_list.append(integers)
            file_names.append(file_name)
    
    return file_list, file_names  

In [6]:
# Insert location point clouds 
PC_location = "../data/input/point clouds"

# Get XY coordinates and file names of the point clouds
PC_XYs, PC_file_names = get_PC_files(PC_location)

In [7]:
def find_PC_files(CWs, PC_XYs, PC_file_names):
    def generate_file_name(x, y):
        return f"filtered_{x}_{y}"
    
    def add_adjacent_files(files, rounded_bounds, minx_dec, miny_dec, maxx_dec, maxy_dec):
        minx_low = rounded_bounds[0] - 1
        miny_low = rounded_bounds[1] - 1
        maxx_high = rounded_bounds[2] + 1
        maxy_high = rounded_bounds[3] + 1

        if minx_dec < 0.04:
            files.append(generate_file_name(minx_low, rounded_bounds[1]))
            files.append(generate_file_name(minx_low, rounded_bounds[3]))

        if miny_dec < 0.04:
            files.append(generate_file_name(rounded_bounds[0], miny_low))
            files.append(generate_file_name(rounded_bounds[2], miny_low))

        if maxx_dec > 0.96:
            files.append(generate_file_name(maxx_high, rounded_bounds[1]))
            files.append(generate_file_name(maxx_high, rounded_bounds[3]))

        if maxy_dec > 0.96:
            files.append(generate_file_name(rounded_bounds[0], maxy_high))
            files.append(generate_file_name(rounded_bounds[2], maxy_high))

        if minx_dec < 0.04 and maxy_dec > 0.96:
            files.append(generate_file_name(minx_low, maxy_high))

        if minx_dec < 0.04 and miny_dec < 0.04:
            files.append(generate_file_name(minx_low, miny_low))

        if maxx_dec > 0.96 and maxy_dec > 0.96:
            files.append(generate_file_name(maxx_high, maxy_high))

        if maxx_dec > 0.96 and miny_dec < 0.04:
            files.append(generate_file_name(maxx_high, miny_low))

        return files

    CWs_dict = []
    PC_list = []

    # Loop over all crosswalks
    for CW in CWs.iterrows():
        # Get the polygon of the crosswalk
        polygon = CW[1]['geometry']
        
        # Get min and max coordinates of polygon
        minx, miny, maxx, maxy = polygon.bounds

        # Divide by 50 to be in accordance with PC file names and round the bounds
        rounded_bounds = [int(minx / 50), int(miny / 50), int(maxx / 50), int(maxy / 50)]
        
        # Generate base file names
        files = [
            generate_file_name(rounded_bounds[0], rounded_bounds[1]),
            generate_file_name(rounded_bounds[0], rounded_bounds[3]),
            generate_file_name(rounded_bounds[2], rounded_bounds[1]),
            generate_file_name(rounded_bounds[2], rounded_bounds[3])
        ]

        # Calculate decimal parts to check for boundary conditions
        minx_dec = minx / 50 - rounded_bounds[0]
        miny_dec = miny / 50 - rounded_bounds[1]
        maxx_dec = maxx / 50 - rounded_bounds[2]
        maxy_dec = maxy / 50 - rounded_bounds[3]

        # Add adjacent files if boundaries are close to the PC border
        files = add_adjacent_files(files, rounded_bounds, minx_dec, miny_dec, maxx_dec, maxy_dec)

        # Remove duplicates by converting to set and back to list
        files = list(set(files))

        CW_dict = {
            'CW_index': CW[0],
            'CW_polygon': polygon,
            'PC_list': files
        }

        CWs_dict.append(CW_dict)
        PC_list.extend(files)
    
    PC_list = list(set(PC_list))

    return CWs_dict, PC_list


In [8]:
CWs, PC_list = find_PC_files(CW_polygons, PC_XYs, PC_file_names)

In [9]:
# Function to load the PC files
def load_PCs(PC_list, folder):
    PCs = []
    
    for pc_name in PC_list:
        file = os.path.join(folder, pc_name + ".laz")
        if os.path.exists(file):
            laz_file = laspy.read(file)
            name = pc_name.split(".")[0]
            PC_coords = laz_file.xyz
            PC_intensity = laz_file.intensity

            PCs.append({"name": name, "laz_file": laz_file, "PC_coords": PC_coords, "PC_intensity": PC_intensity})
        else: 
            print(file, "does not exist")
           
    return PCs

In [10]:
# Load the PC files into a dictionary
PCs = load_PCs(PC_list, PC_location)

# Down sampling and cutting point clouds

As we are only interested in points that are on the ground, we can cut points above a certain threshold from the point clouds. Doing so will speed up further processing. 
 Additionally, we will donwsample the point clouds. This is because we do not need the information from every point in the point cloud in order to obtain good results. Down sampling will also improve the processing time.

In [11]:
# Function to cut points in the point clouds that are above a certain threshold
def cut_PC(pc):

    # Find coordinates below threshold
    indices = np.where(pc['PC_coords'][:, 2] < 5)

    # Cut intensity accordingly
    pc['PC_intensity_low'] = pc['PC_intensity'][indices]
    pc['PC_coords_low'] = pc['PC_coords'][indices]

    # Take the mean
    mean = np.mean(pc['PC_coords_low'][:, 2])

    # Cut again based on one std away from the mean
    std_deviation = np.std(pc['PC_coords_low'][:, 2])

    # Determine new threshold 
    threshold = mean + std_deviation

    # Find coordinates below new threshold
    indices_new = np.where(pc['PC_coords_low'][:, 2] < threshold)

    # Cut intensity accordingly
    pc['PC_intensity_low'] = pc['PC_intensity_low'][indices_new]
    pc['PC_coords_low'] = pc['PC_coords_low'][indices_new]

    return pc

In [12]:
# Function to downsample the PCs
def down_sample_PC(pc, coords, intensity_string):
    xyz = pc[coords]
    intensity = pc[intensity_string]
    
    # Convert to Open3D point cloud using only XYZ
    pc_o3d = o3d.geometry.PointCloud()
    pc_o3d.points = o3d.utility.Vector3dVector(xyz)

    # Perform voxel downsampling
    downsampled_pc_o3d = pc_o3d.voxel_down_sample(0.02)

    # Retrieve downsampled XYZ points
    pc[coords + "_ds"] = np.asarray(downsampled_pc_o3d.points) 

    # Create a KDTree for the original point cloud
    tree = cKDTree(xyz)

    # For each downsampled point, find its nearest neighbor in the original cloud
    _, indices = tree.query(pc[coords + '_ds'])

    # Get indices of intensity
    pc[intensity_string + "_ds"] = intensity[indices]

    return pc

In [13]:
# Function to process the PCs
def cut_ds_PC(PCs, coord_string, intensity_string):
    PCs_cut = []

    for pc in PCs:
        pc = cut_PC(pc)
        pc = down_sample_PC(pc, coord_string, intensity_string)
        PCs_cut.append(pc)
    
    return PCs_cut

In [14]:
# Downsample and cut the polygons based on height
PCs_cut = cut_ds_PC(PCs, "PC_coords_low", "PC_intensity_low")
del PCs

# Matching CW polygons to PCs

In [15]:
# Cut point clouds based on polygon coordinate
def PC_pol_match(PC, pol):

    # Get the bounding box (rectangle) of the polygon
    minx, miny, maxx, maxy = pol['CW_polygon'].bounds
    
    # Determine condition based on polygon bounds
    condition = ((PC['PC_coords_low_ds'][:, 0] > minx) & (PC['PC_coords_low_ds'][:, 0] < maxx) 
                &  (PC['PC_coords_low_ds'][:, 1] > miny) & (PC['PC_coords_low_ds'][:, 1] < maxy))

    # Apply condition to get indices
    indexes = np.where(condition)

    # Check if any matches were found
    if len(indexes[0]) > 0:
        
        # Apply indexing to coordinates and intensity
        intensity = PC['PC_intensity_low_ds'][indexes]
        coords = PC['PC_coords_low_ds'][indexes]

        return {'CW_index': pol['CW_index'], 'polygon': pol['CW_polygon'], 'PC_file': [PC['name']], 'PC_coords_low_ds': coords, 'PC_intensity_low_ds': intensity}

In [16]:
# Function to merge polygons that are spread over two point clouds
def merge_matches(match1, match2):

    # Concatenate coordinates and intensity of both PC files belonging to the same polygon 
    coords = np.vstack((match1['PC_coords_low_ds'], (match2['PC_coords_low_ds'])))
    intensity = np.hstack((match1['PC_intensity_low_ds'], (match2['PC_intensity_low_ds'])))
    
    # Create list of PC files to add to dictionary 
    PC_list = match1['PC_file'] + match2['PC_file']
    
    # Create dictionary for matched point clouds
    new_match = {'CW_index': match1['CW_index'], 'polygon': match1['polygon'], 'PC_file': PC_list, 'PC_coords_low_ds': coords, 'PC_intensity_low_ds': intensity}
    
    return new_match

In [17]:
# Function to group PC matches of the same polygon together
def group_matches(all_matches):
    # Create list to group together polygons that are spread over multiple point clouds
    grouped_data = []

    # Create a deep copy of the previously identified matches
    match_copy = copy.deepcopy(all_matches)

    # Loop over all matches
    for item in match_copy:

        index = item['CW_index']

        found = False

        for sublist in grouped_data:

            # Check if the polygon is already in the list and append to the corresponding list item if this is the case
            if sublist and sublist[0]['CW_index'] == index:
                sublist.append(item)
                found = True
                break
            
        # If the polygon is not already in the list, append it 
        if not found:
            grouped_data.append([item])
    
    return grouped_data

In [18]:
# Function to process PC matches of the same polygon
def process_grouped_matches(grouped_data):
    # Loop over the grouped polygons
    for group in grouped_data:

        # Check if there is multiple PC files for one polygon 
        if len(group) > 1:

            # Loop over each item except the last one
            for i in range(len(group) - 1):

                # Merge the first item with the next one and replace the first item 
                match = merge_matches(group[0], group[1])
                
                group[0] = match

                group.pop(1)

    # Flatten the grouped data list as each list item only has one item now
    grouped_data_flat = [item for sublist in grouped_data for item in sublist]

    return grouped_data_flat

In [19]:
# Function to match PCs and polygons
def match_PC_pol(CW_polygons, PCs):

    # Create list to save all matches found
    all_matches = []

    # Loop over all polygons
    for index in range(0, len(CW_polygons)):
        cw = CW_polygons[index]

        for pc in PCs:
            if pc['name'] in cw['PC_list']:
                match = PC_pol_match(pc, cw)
                if match:
                    all_matches.append(match)
    
    grouped_data = group_matches(all_matches)
    merged_data = process_grouped_matches(grouped_data)
    
    return merged_data, grouped_data, all_matches

In [20]:
# Find the PC data that matches the polygons
merged_data, grouped_data, all_matches = match_PC_pol(CWs, PCs_cut)

# Growing polygons

To filter the polygons, we take several steps.
1. Cluster areas within a polygon with a high intensity value in close proximity together 
2. For each cluster, check the surrounding points. If points have a high intensity value, add them to the cluster.

Step 2 is repeated untill there are no more surrounding points with a high intensity value. This way, only areas that are present in the original polygon are grown and outside areas are not included.

In [21]:
# Function to filter the PC points in a polygon based on the intensity values of the points
def filter_intensity(polygon, og_intensity, og_coords, new_intensity, new_coords):
    return_polygon = copy.deepcopy(polygon)

    # Determine condition based on polygon bounds
    condition = (return_polygon[og_intensity] > 26000)

    # Apply condition to get indices
    indexes = np.where(condition)

    # Check if any matches were found
    if len(indexes[0]) > 0:
        
        # Apply indexing to coordinates and intensity
        return_polygon[new_intensity] = return_polygon[og_intensity][indexes]
        return_polygon[new_coords] = return_polygon[og_coords][indexes]
       

        return return_polygon

In [22]:
# Function to cluster the polygon into clusters of points that have a high intensity and are close together in space
def cluster_pol(CW):
    # Create list to save the clusters that are found
    cluster_list = []

    # Filter the original polygon to only include points with a high intensity
    filtered = filter_intensity(CW, "PC_intensity_low_ds", "PC_coords_low_ds", "PC_intensity_low_ds_filtered", "PC_coords_low_ds_filtered")
    

    if filtered:
    
        # Use DBSCAN to cluster the points in the polygon
        dbscan = DBSCAN(eps=0.1, min_samples=5)
        dbscan.fit(filtered['PC_coords_low_ds_filtered'])

        # Get labels created by DBSCAN
        labels = dbscan.labels_

        # Create dictionary to save clusters
        cluster_data = {}

        # Loop over each point in the filtered polygon and check to which cluster it belongs
        # Group coordinates and intensity values based on their label in the cluster_data dictionary
        for label, point, value in zip(labels, filtered['PC_coords_low_ds_filtered'], filtered['PC_intensity_low_ds_filtered']):
            if label not in cluster_data:
                cluster_data[label] = {'coords': [], 'intensity': []}  
            cluster_data[label]['coords'].append(point)
            cluster_data[label]['intensity'].append(value)
        
        # Transform the coordinates and intensity values to np arrays to make them easier to work with
        for label in np.unique(labels):
            cluster_data[label]['coords'] = np.array(cluster_data[label]['coords'])
            cluster_data[label]['intensity'] = np.array(cluster_data[label]['intensity'])

        # Loop over the created clusters and save them in the cluster_list
        for cluster in cluster_data:
            cluster_dict = {}

            # Only keep clusters that are over 100 points to pre-emptively filter out noise
            if (len(cluster_data[cluster]['coords']) > 50):

                # Save cluster in a similar manner as the original polygon
                cluster_dict['CW_index'] = CW['CW_index']
                cluster_dict['PC_file'] = CW['PC_file']
                cluster_dict['coordinates'] = cluster_data[cluster]['coords']
                cluster_dict['intensity'] = cluster_data[cluster]['intensity']
                
                cluster_list.append(cluster_dict)
        
        return cluster_list

In [23]:
# Function to grow the clusters based on creating a buffer around them and seeing if there are points in the buffer with a high intensity
def grow_cluster(PC_coords, PC_intensity, cluster_coords):

    # Initialize an empty list to keep track of the points that are added to the clusters
    added = []

    # Build a KDTree for fast nearest neighbor search
    tree = cKDTree(PC_coords)

    # Define the radius within which points are considered neighbors
    radius = 0.12

    # Initialize the starting coordinates for the cluster growth as the original cluster
    coords = cluster_coords

    while True:

        # Find indices of neighbors within the specified radius
        neighbor_indices = tree.query_ball_point(coords, radius)

        # Initialize a list to store unique inidces of new points to add
        indices = []

        # Iterate through the neighbor indices to see if points have already been added
        for index in neighbor_indices:
            for i in index:
                # Add them if this is not the case
                if i not in added:
                    indices.append(i)
                    added.append(i)

        # Remove duplicates from the lists of indices
        indices = list(set(indices))
        added = list(set(added))

        # If no new points are found, exit the loop
        if len(indices) == 0:
            break

        # Retrieve coordinates and intensities of the neighboring points
        neighbor_coords = PC_coords[indices]
        neighbor_intensities = PC_intensity[indices]

        # Store the neighbors in a temporary dictionary
        temp = {'coords': neighbor_coords, 'intensity': neighbor_intensities}

        # Apply a filtering function to the temporary dictionary to only keep points with a high intensity
        temp_filtered = filter_intensity(temp, "intensity", "coords", "intensity_filtered", "coords_filtered")
        
        if temp_filtered:
            # If new filtered coordinates are available, update the coordinates for the next iteration
            if 'coords_filtered' in temp_filtered:
                coords = temp_filtered['coords_filtered']

    # Extract the final cluster coordinates and intensities
    cluster_coords = PC_coords[added]
    cluster_intensity = PC_intensity[added]

    # Store the final cluster information in a dictionary
    final = {'coords': cluster_coords, 'intensity': cluster_intensity}

    # Apply filtering to the final cluster
    final = filter_intensity(final, "intensity", "coords", "intensity_filtered", "coords_filtered")
    
    # Return the filtered final cluster
    return final


In [24]:
# Function to create and process the PC polygon points
def get_clusters(polygon, PCs, PC_coords_string, PC_intensity_string): 

    # Initialize coordinate and intensity array
    PC_coords_temp = []
    PC_intensity_temp = []

    # Get PC file that corresponds to that of the original polygon
    for PC_name in polygon['PC_file']:
        PC = list(filter(lambda PC: PC['name'] == PC_name, PCs))
        
        sub_PC_coords = PC[0][PC_coords_string]
        sub_PC_intensity = PC[0][PC_intensity_string]
        
        PC_coords_temp.append(sub_PC_coords)
        PC_intensity_temp.append(sub_PC_intensity)
    
    
    PC_coords = np.concatenate(PC_coords_temp, axis=0)
    PC_intensity = np.concatenate(PC_intensity_temp, axis=0)

    # Get clusters from polygon
    cluster_dict = cluster_pol(polygon)

    if cluster_dict:

        # For each found cluster, grow it and update the cluster data
        for cluster in cluster_dict:
            clean_cluster = grow_cluster(PC_coords, PC_intensity, cluster['coordinates'])
            if 'coords_filtered' in clean_cluster:
                cluster['clean_coords'] = clean_cluster['coords_filtered']
                cluster['clean_intensity'] = clean_cluster['intensity_filtered']
        
        # Return cluster dictionary
        return cluster_dict

In [27]:
def get_cluster_dict(merged_data, polygon_origin):

    # Set path to save cluster dictionary based on polygons
    if polygon_origin == "T2N":
        path = "../data/output/cluster dict.pkl"
    if polygon_origin == "extension":
        path = "../data/output/extension cluster dict.pkl"

    # Cluster the PC polygons and grow them to get complete road markings
    final = []
    too_long = []
    for merge in merged_data:
        try:
            with timeout(3600, exception=RuntimeError):

                print("working on", merge['CW_index'])
                cluster_dict = get_clusters(merge, PCs_cut, 'PC_coords_low_ds', 'PC_intensity_low_ds')

                if cluster_dict:
                    print("found clusters for", merge['CW_index'])
                    final.append(cluster_dict)

                    with open(path, 'wb') as file:
                        pickle.dump(final, file)
                else:
                    print("did not find clusters")

        except RuntimeError:
            print(merge['CW_index'], "took too long")
            too_long.append(merge['CW_index'])


In [28]:
get_cluster_dict(merged_data, polygon_origin)